# Expedia hotel recommendations with Apache Spark

The size of the train set is 3.8 GB, with more than 37 million samples! That is why I will be using Pyspark instead of our favorite Pandas! When possible for visualization I will be using toPandas() command to convert pyspark dataframes to pandas. 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
from pyspark.sql.functions import count
import seaborn as sns

In [2]:
spark = SparkSession.builder.getOrCreate()

## 1. Import and Explore

### 1.1. Import:

In [3]:
train = spark.read.csv('../data/input/train.csv', header=True)
test = spark.read.csv('../data/input/test.csv', header=True)

### 1.2 Explore

What attriutes we have?

In [4]:
train.printSchema()

root
 |-- date_time: string (nullable = true)
 |-- site_name: string (nullable = true)
 |-- posa_continent: string (nullable = true)
 |-- user_location_country: string (nullable = true)
 |-- user_location_region: string (nullable = true)
 |-- user_location_city: string (nullable = true)
 |-- orig_destination_distance: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- is_mobile: string (nullable = true)
 |-- is_package: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: string (nullable = true)
 |-- srch_children_cnt: string (nullable = true)
 |-- srch_rm_cnt: string (nullable = true)
 |-- srch_destination_id: string (nullable = true)
 |-- srch_destination_type_id: string (nullable = true)
 |-- is_booking: string (nullable = true)
 |-- cnt: string (nullable = true)
 |-- hotel_continent: string (nullable = true)
 |-- hotel_country: string (nullable = true)
 

In [5]:
test.printSchema()

root
 |-- id: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- site_name: string (nullable = true)
 |-- posa_continent: string (nullable = true)
 |-- user_location_country: string (nullable = true)
 |-- user_location_region: string (nullable = true)
 |-- user_location_city: string (nullable = true)
 |-- orig_destination_distance: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- is_mobile: string (nullable = true)
 |-- is_package: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: string (nullable = true)
 |-- srch_children_cnt: string (nullable = true)
 |-- srch_rm_cnt: string (nullable = true)
 |-- srch_destination_id: string (nullable = true)
 |-- srch_destination_type_id: string (nullable = true)
 |-- hotel_continent: string (nullable = true)
 |-- hotel_country: string (nullable = true)
 |-- hotel_market: string (nullable = true)


How many samples does the train data include?

In [6]:
n_train = train.count()
print(n_train)

37670293


Quite big! Around 38 millions!

How many samples does the test data include? 

In [7]:
test.count()

2528243

**First Visualization**:

To see the first five rows of the train set, it is possible to use train.head(5) or train.show(5) (slightly better visualization), but still it can be quite hard to get a sense of the data. Instead, I use Pandas:

In [8]:
pd.DataFrame(train.head(5), columns=train.columns)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


Similarly, for .describe():

In [9]:
train.describe().toPandas()

,summary,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,count,37670293,37670293,37670293,37670293,37670293,37670293,24145292,37670293,37670293,...,37670293,37670293,37670293,37670293,37670293,37670293,37670293,37670293,37670293,37670293
1,mean,None,9.795271329585889,2.6804730188851997,86.10880194109454,308.4060117610447,27753.044729330883,1970.0900267207285,604451.7531778422,0.1349265056154461,...,0.3321221579030458,1.1126628083301608,14441.090543760836,2.5822799148389954,0.07965674702875288,1.4833839227106622,3.1563047837190967,81.29685165974153,600.461883638654,49.80860501934509
2,stddev,None,11.9675435665128,0.7480393482506577,59.24310334783878,208.44374973856722,16782.553195680346,2232.4424303904275,350617.4620408585,0.34164505966916764,...,0.7314980986397146,0.45911549963856946,11066.302332627309,2.153018959399955,0.2707610600283715,1.2197755786558424,1.6231886782105662,56.171188062887815,511.73912727922396,28.915950805004293
3,min,2013-01-07 00:00:02,10,0,0,0,0,0.0056,0,0,...,0,0,0,0,0,1,0,0,0,0
4,max,2014-12-31 23:59:59,9,4,99,999,9998,9999.9984,999999,1,...,9,8,9999,9,1,99,6,99,999,99


What are the feature types? 

In [10]:
train.dtypes

[('date_time', 'string'),
 ('site_name', 'string'),
 ('posa_continent', 'string'),
 ('user_location_country', 'string'),
 ('user_location_region', 'string'),
 ('user_location_city', 'string'),
 ('orig_destination_distance', 'string'),
 ('user_id', 'string'),
 ('is_mobile', 'string'),
 ('is_package', 'string'),
 ('channel', 'string'),
 ('srch_ci', 'string'),
 ('srch_co', 'string'),
 ('srch_adults_cnt', 'string'),
 ('srch_children_cnt', 'string'),
 ('srch_rm_cnt', 'string'),
 ('srch_destination_id', 'string'),
 ('srch_destination_type_id', 'string'),
 ('is_booking', 'string'),
 ('cnt', 'string'),
 ('hotel_continent', 'string'),
 ('hotel_country', 'string'),
 ('hotel_market', 'string'),
 ('hotel_cluster', 'string')]

The read.csv() function of pyspark read all the features as string! So, we convert them appropriately:

In [12]:
print(train.columns, "\n")
print("The train set has", len(train.columns), "columns.")

['date_time', 'site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster'] 

The train set has 24 columns.


In [13]:
int_columns = ['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 
               'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
               'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 
               'hotel_market', 'hotel_cluster']
float_columns = ['orig_destination_distance']
date_columns = ['date_time', 'srch_ci', 'srch_co']

In [14]:
for i in range(len(int_columns)):
    train = train.withColumn(int_columns[i], train[int_columns[i]].astype('int'))
for i in range(len(float_columns)):
    train = train.withColumn(float_columns[i], train[float_columns[i]].astype('float'))

Let's now check the types:

In [15]:
train.dtypes

[('date_time', 'string'),
 ('site_name', 'int'),
 ('posa_continent', 'int'),
 ('user_location_country', 'int'),
 ('user_location_region', 'int'),
 ('user_location_city', 'int'),
 ('orig_destination_distance', 'float'),
 ('user_id', 'int'),
 ('is_mobile', 'int'),
 ('is_package', 'int'),
 ('channel', 'int'),
 ('srch_ci', 'string'),
 ('srch_co', 'string'),
 ('srch_adults_cnt', 'int'),
 ('srch_children_cnt', 'int'),
 ('srch_rm_cnt', 'int'),
 ('srch_destination_id', 'int'),
 ('srch_destination_type_id', 'int'),
 ('is_booking', 'int'),
 ('cnt', 'int'),
 ('hotel_continent', 'int'),
 ('hotel_country', 'int'),
 ('hotel_market', 'int'),
 ('hotel_cluster', 'int')]

** How many are really booking? **

In [16]:
train.where(train.is_booking == 1).count()

3000693

Only around 3 millions of the 38 millions data 'booked', and the rest where only a click! So chance of booking when click is:

In [17]:
print(train.where(train.is_booking == 1).count() / train[train.is_booking.isNotNull()].count() * 100, '%')

7.965674702875288 %


So, only **8%** of the train data is booking and the rest are clicks! 

In [18]:
print("train data summary:\n",
      train.select('user_location_country').distinct().count(), "user countries\n", 
      train.select('user_location_city').distinct().count(), "user cities\n", 
      train.select('hotel_country').distinct().count(), "hotel countries\n",
      train.select('hotel_cluster').distinct().count(), "hotel clusters\n",
      train.select('srch_destination_type_id').distinct().count(), "search destination types\n", 
      train.select('user_id').distinct().count(), "distinct users")

train data summary:
 237 user countries
 50447 user cities
 213 hotel countries
 100 hotel clusters
 10 search destination types
 1198786 distinct users


So only ~1.2 millions users generate the ~38 millions logs!

Are there any users in common in train-test sets? 

Since there exists only 1.2 different users and average cnt (number of clicks in a Session) is only ~1.5, that would mean that the logs of the same users has been recorded in different sessions or different srch_destination_ids. 

In [20]:
pd.DataFrame(train.head(5), columns=train.columns)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.264160,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.264160,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.264160,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.193176,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.625916,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [21]:
train.select('is_booking').rdd.map(lambda row : row.is_booking).take(30);

In [22]:
test.describe().toPandas()

,summary,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,count,2528243,2528243,2528243,2528243,2528243,2528243,2528243,1680782,2528243,...,2528222,2528226,2528243,2528243,2528243,2528243,2528243,2528243,2528243,2528243
1,mean,1264121.0,None,9.70966556616591,2.697412788248598,85.51611929707705,317.0340683233376,27862.17806318459,1686.3979638184105,602645.9257690024,...,None,None,1.958126651591639,0.25074132510205704,1.117409995795499,16568.621168139296,2.941701806353266,3.022614914784694,81.25454277931354,646.4925847713214
2,stddev,729841.0326509465,None,12.271834283602896,0.7651287624994623,56.73879902333156,208.28338881273413,16687.877645606164,2197.361433047482,346965.11778565275,...,None,None,0.8980415286253776,0.6754566222733871,0.4615009555032746,12805.019853860984,2.282005929487735,1.6748793299399136,55.07528338943058,504.65298547297846
3,min,0,2015-01-08 00:00:35,0,0,0,0,0,0.0056,0,...,2015-01-04,2015-01-08,0,0,0,10,1,0,0,0
4,max,999999,2015-12-31 23:58:47,9,4,99,999,9999,9999.9851,999999,...,2161-10-00,2017-04-30,9,9,8,9999,9,6,99,999


### 1.3 Missing data

## 2. Feature engineering

For Exploratory Data Analysis (EDA), working with the large data, and especially visuallizing the data is almost impossible. Instead we will be using a random sample of 1 % of the data for EDA. 

In [24]:
sample_train_pd = train.sample(False, 0.01).toPandas()
# only the users with is_booking = 1
sample_trainB_pd = train.where(train.is_booking == 1).sample(False, 0.1).toPandas()
# users with is_booking = 0
sample_trainNB_pd = train.where(train.is_booking == 0).sample(False, 0.01).toPandas()

In [25]:
test_pd = test.sample(False, 0.5).toPandas()

The size of the sampled train data is:

In [26]:
sample_train_pd.shape

(376911, 24)

In [27]:
print("The ratio of booking to all in the sampled train data is", len(sample_train_pd.loc[sample_train_pd['is_booking'] == 1]) / len(sample_train_pd) * 100, " %.")

The ratio of booking to all in the sampled train data is 7.9334378672949315  %.


In [28]:
print("The ration of booking to all in all train data is", train.where(train.is_booking == 1).count() / train[train.is_booking.isNotNull()].count() * 100, '%.')

The ration of booking to all in all train data is 7.965674702875288 %.


In [ ]:
sns.barplot(x = 'is_package', y = 'is_booking', data=sample_train_pd)

In [ ]:
plt.subplots(1, 2, figsize = (15, 5))
plt.subplot(1, 2, 1)
plt.hist(sample_trainB_pd['hotel_cluster'], bins = range(0, 99), density = True)
plt.title('Only clicks data')
plt.xlabel('Hotel Clusters')
plt.subplot(1, 2, 2)
plt.hist(sample_trainNB_pd['hotel_cluster'], bins = range(0, 99), density= True)
plt.title('Only booking data')
plt.xlabel('Hotel Clusters')

The distribution of the selected hotel clusters in the only-book-data is different from only-click-data. Given that the test data is only on booking data, the question is should we only use only-book-data? To better answer this question, we do a more systematic comparison between only-book and only-click data.

In [ ]:
hotel_cluster_hist_B = np.histogram(sample_trainB_pd['hotel_cluster'], bins = range(99), density=True)
hotel_cluster_hist_NB = np.histogram(sample_trainNB_pd['hotel_cluster'], bins = range(99), density = True)

In [ ]:
hotel_cluster_hist_B

In [ ]:
plt.plot(100 * (hotel_cluster_hist_B[0] - hotel_cluster_hist_NB[0]) / hotel_cluster_hist_B[0], '.-')
plt.title('Percentage difference between only-clicks and only-book clusters')

In [ ]:
clusterB = sample_trainB_pd['hotel_cluster'].value_counts(normalize = True).sort_index()
clusterNB = sample_trainNB_pd['hotel_cluster'].value_counts(normalize = True).sort_index()

plt.subplots(1, 2, figsize = (15, 5))
plt.subplot(1, 2, 1)
plt.plot(clusterB, '.-')
plt.plot(clusterNB, '.-')
plt.subplot(1, 2, 2)
plt.plot((clusterNB - clusterB) / clusterB * 100, '.-')

In [ ]:
for i in range(len(test_pd['user_id'])):
    for j in range(len(sample_train_pd['user_id'])):
        if (test_pd['user_id'][i] == sample_train_pd['user_id'][j]):
            print(i, j)

In [ ]:
trainB = train.where(train.is_booking == 1)
# Equivalently I could use the following statement SQL query
# sqlContext.registerDataFrameAsTable(df = train, tableName="train")
# trainB = sqlContext.sql("SELECT * FROM train WHERE is_booking == 1")

In [ ]:
trainB.count()

In [ ]:
train_cnt_array = train.select('cnt').rdd.map(lambda row : row.cnt).sample(False, 0.0001).collect()

## 3. ML models and predictions

In [ ]:
train_sp = train_sp.withColumn('is_booking', train_sp['is_booking'].astype("float"))

In [ ]:
train_sp[['is_booking']].describe().toPandas()

In [ ]:
train_sp.where(col('is_booking').isNull())

In [ ]:
train_sp.columns

In [ ]:
test = pd.read_csv('../data/input/test.csv')

In [ ]:
chunksize = 1 * 10 ** 6
train_reader = pd.read_csv('../data/input/train.csv', chunksize=chunksize)

In [ ]:
all_chunks = []
i = 0
for chunk in train_reader:
    all_chunks.append(chunk)
    print(i)
    i += 1

In [ ]:
train = pd.DataFrame()

In [ ]:
import time

In [ ]:
for i in range(5, 10):
    start_time = time.time()
    train = pd.concat(objs = [train, all_chunks[i]], axis = 0).reset_index(drop = True)
    end_time = time.time()
    print(i)
    print(end_time - start_time)

In [ ]:
train.head()